In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv("Phishing_Email.csv", encoding = "latin-1")

X_data = data['Email Text']
y = data['Email Type']

In [2]:
import nltk
# nltk.download()

In [3]:
print(X_data[6])

On Sun, Aug 11, 2002 at 11:17:47AM +0100, wintermute mentioned:
> > The impression I get from reading lkml the odd time is
> > that IDE has gone downhill since Andre Hedrick was 
> > effectively removed as maintainer. Martin Dalecki seems
> > to have been unable to further development without 
> > much breakage. 
> 
> Hmm... begs the question, why remove Handrick?
> If it ain't broke, don't fix it. See, the IDE subsystem is like the One Ring. It's kludginess, due to
having to support hundreds of dodgy chipsets & drives means that it is
inherintly evil. A few months of looking at the code can turn you sour.
Years of looking at it will turn you into an arsehole. They haven't found a hobbit that can code, so mortal humans have to
suffice. Kate
-- 
Irish Linux Users' Group: ilug@linux.ie
http://www.linux.ie/mailman/listinfo/ilug for (un)subscription information.
List maintainer: listmaster@linux.ie



In [4]:
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import re

def clean_text(text):    
  if not isinstance(text, str):
    text = str(text)  # to string
    
  number_pattern = r"\d+"
  text = re.sub(number_pattern, "", text)  # remove numbers
    
  url_pattern = r"http[s]?://\S+"  # remove URLs
  text = re.sub(url_pattern, "", text)
    
  text = text.lower()

  text = text.translate(str.maketrans('', '', string.punctuation)) #removes punctuation

  stop_words = stopwords.words('english')
  text = ' '.join([word for word in text.split() if word not in stop_words]) #removes stopwords

  lemmatizer = WordNetLemmatizer()
  text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

  return text

clean_X_data = []
for text in X_data:
  clean_X_data.append(clean_text(text))

In [38]:
print(clean_X_data[6])

sun aug wintermute mentioned impression get reading lkml odd time ide gone downhill since andre hedrick effectively removed maintainer martin dalecki seems unable development without much breakage hmm begs question remove handrick aint broke dont fix see ide subsystem like one ring kludginess due support hundred dodgy chipsets drive mean inherintly evil month looking code turn sour year looking turn arsehole havent found hobbit code mortal human suffice kate irish linux user group iluglinuxie unsubscription information list maintainer listmasterlinuxie


In [5]:
data['Email Text'] = clean_X_data
data.to_csv("Phishing_Email.csv", index=False)

In [5]:
print(clean_X_data[0])

disc uniformitarianism sex lang dick hudson observation u use aughter vocative thoughtprovoking sure fair attribute son treated like senior relative one thing nt normally use brother way aughter hard imagine natural class comprising senior relative excluding brother another seem difference imagining distinction seems senior relative term used wider variety context e g calling distance get someone attention hence beginning utterance whereas seems natural utterance like yes son hand son one like son son help although perhaps latter one completely impossible alexis mr


In [39]:
from random import random
from collections import Counter

def balance_data(X_data, y, target_label="Safe Email", keep_proportion=0.64):
  class_counts = Counter(y)
  safe_count = class_counts[target_label]

  # keep 64% of the Safe Emails
  undersampled_data = []
  undersampled_labels = []
  for i, (data_point, label) in enumerate(zip(X_data, y)):
    if label == target_label:
      if random() < keep_proportion:
        undersampled_data.append(data_point)
        undersampled_labels.append(0)
    else:
      # keep all Phishing Emails
      undersampled_data.append(data_point)
      undersampled_labels.append(1)

  return undersampled_data, undersampled_labels

balanced_X, balanced_y = balance_data(clean_X_data, y)

In [40]:
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split

ngrams = (1, 1) ####

vectorizer = CountVectorizer(min_df=2, lowercase=True, ngram_range=ngrams, stop_words='english', max_features=500)
X_data_counts = vectorizer.fit_transform(balanced_X)

tf_transformer = TfidfTransformer(use_idf=True).fit(X_data_counts)
X_data_tf = tf_transformer.transform(X_data_counts)

X_train, X_test, y_train, y_test = train_test_split(X_data_tf, balanced_y, test_size=0.2, random_state=42)

classifier = MultinomialNB()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

In [46]:
print(X_train)

  (1, 495)	0.31536122508848546
  (1, 493)	0.14066204771452306
  (1, 474)	0.16284801364615373
  (1, 468)	0.1485320800529444
  (1, 467)	0.1191579774798663
  (1, 464)	0.17335113816830258
  (1, 455)	0.17568281854700413
  (1, 449)	0.10135120786085484
  (1, 433)	0.1571791952861873
  (1, 407)	0.3289766247587358
  (1, 405)	0.16585965676641837
  (1, 353)	0.1809362349558374
  (1, 348)	0.1770447604482475
  (1, 344)	0.15672852005560317
  (1, 306)	0.1405401168506235
  (1, 293)	0.36013146165208876
  (1, 292)	0.14879081335437697
  (1, 290)	0.1803013497968239
  (1, 275)	0.17355172491585452
  (1, 244)	0.12869163323221472
  (1, 232)	0.10503903406690276
  (1, 231)	0.17120593684375088
  (1, 163)	0.13175926024607654
  (1, 158)	0.16347535925946882
  (1, 136)	0.17409137442512712
  :	:
  (11643, 236)	0.2953981255145133
  (11643, 224)	0.10674200688194156
  (11643, 216)	0.08899974309643179
  (11643, 199)	0.07408513379766374
  (11643, 190)	0.05095213092092492
  (11643, 174)	0.3666312352919519
  (11643, 157)	0.06

In [8]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_auc_score, f1_score

def topTenWords(vocabulary):
    indices = np.where(y_pred == 1)[0]

    predicted_phishing_emails = X_test[indices]

    alpha = 0.1
    
    class_log_probs = classifier.predict_log_proba(predicted_phishing_emails)[:, 1]
    phishing_weights = [prob - np.log(alpha) for prob in class_log_probs]
    phishing_features = vocabulary

    sorted_features = sorted(zip(phishing_features, phishing_weights), key=lambda x: x[1], reverse=True)

    top_10_phishing_words = [word for word, _ in sorted_features[:10]]

    print("Top 10 words for predicted phishing emails (considering Laplace smoothing):", top_10_phishing_words)

        
        
def displayStats():
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)
    precision = precision_score(y_test, y_pred)
    print("Precision:", precision)
    recall = recall_score(y_test, y_pred)
    print("Recall:", recall)
    f1 = f1_score(y_test, y_pred)
    print("F1-measure:", f1)
    roc_auc = roc_auc_score(y_test, y_pred)
    print("ROC AUC score:", roc_auc)
    confusion_matrix_result = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix:\n", confusion_matrix_result)
    
        
topTenWords(vectorizer.vocabulary_)
displayStats()

Top 10 words for predicted phishing emails (considering Laplace smoothing): ['total', 'id', 'friend', 'energy', 'ect', 'second', 'support', 'present', 'follow', 'fax']
Accuracy: 0.9156749404964298
Precision: 0.9275158339197748
Recall: 0.9008885850991114
F1-measure: 0.9140083217753121
ROC AUC score: 0.9155999082464434
Confusion Matrix:
 [[1375  103]
 [ 145 1318]]


In [9]:
ngrams = (1, 5) ####

vectorizer = CountVectorizer(min_df=2, lowercase=True, ngram_range=ngrams, stop_words='english', max_features=500)
X_data_counts = vectorizer.fit_transform(balanced_X)

tf_transformer = TfidfTransformer(use_idf=True).fit(X_data_counts)
X_data_tf = tf_transformer.transform(X_data_counts)

X_train, X_test, y_train, y_test = train_test_split(X_data_tf, balanced_y, test_size=0.2, random_state=42)

classifier = MultinomialNB()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

In [10]:
topTenWords(vectorizer.vocabulary_)
displayStats()

Top 10 words for predicted phishing emails (considering Laplace smoothing): ['original message', 'pay', 'opportunity', 'professional', 'click', 'complete', 'follow', 'ect', 'america', 'present']
Accuracy: 0.9156749404964298
Precision: 0.9245283018867925
Recall: 0.9043062200956937
F1-measure: 0.9143054595715272
ROC AUC score: 0.915617250778564
Confusion Matrix:
 [[1370  108]
 [ 140 1323]]


In [11]:
ngrams = (1, 1)

vectorizer = CountVectorizer(min_df=2, lowercase=True, ngram_range=ngrams, stop_words='english', max_features=2000) ####
X_data_counts = vectorizer.fit_transform(balanced_X)

tf_transformer = TfidfTransformer(use_idf=True).fit(X_data_counts)
X_data_tf = tf_transformer.transform(X_data_counts)

X_train, X_test, y_train, y_test = train_test_split(X_data_tf, balanced_y, test_size=0.2, random_state=42)

classifier = MultinomialNB()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

In [12]:
topTenWords(vectorizer.vocabulary_)
displayStats()

Top 10 words for predicted phishing emails (considering Laplace smoothing): ['corp', 'cash', 'functional', 'tm', 'quite', 'difference', 'private', 'reading', 'pre', 'term']
Accuracy: 0.9357361441686501
Precision: 0.9556509298998569
Recall: 0.9131920710868079
F1-measure: 0.9339391821041594
ROC AUC score: 0.9356217459628897
Confusion Matrix:
 [[1416   62]
 [ 127 1336]]


In [13]:
ngrams = (1, 1)

vectorizer = CountVectorizer(min_df=10, lowercase=True, ngram_range=ngrams, stop_words='english', max_features=500) ####
X_data_counts = vectorizer.fit_transform(balanced_X)

tf_transformer = TfidfTransformer(use_idf=True).fit(X_data_counts)
X_data_tf = tf_transformer.transform(X_data_counts)

X_train, X_test, y_train, y_test = train_test_split(X_data_tf, balanced_y, test_size=0.2, random_state=42)

classifier = MultinomialNB()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

In [14]:
topTenWords(vectorizer.vocabulary_)
displayStats()

Top 10 words for predicted phishing emails (considering Laplace smoothing): ['total', 'id', 'friend', 'energy', 'ect', 'second', 'support', 'present', 'follow', 'fax']
Accuracy: 0.9156749404964298
Precision: 0.9275158339197748
Recall: 0.9008885850991114
F1-measure: 0.9140083217753121
ROC AUC score: 0.9155999082464434
Confusion Matrix:
 [[1375  103]
 [ 145 1318]]


In [15]:
ngrams = (1, 1)

vectorizer = CountVectorizer(min_df=2, lowercase=True, ngram_range=ngrams, stop_words='english', max_features=100000) ####
X_data_counts = vectorizer.fit_transform(balanced_X)

tf_transformer = TfidfTransformer(use_idf=True).fit(X_data_counts)
X_data_tf = tf_transformer.transform(X_data_counts)

X_train, X_test, y_train, y_test = train_test_split(X_data_tf, balanced_y, test_size=0.2, random_state=42)

classifier = MultinomialNB()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

In [16]:
topTenWords(vectorizer.vocabulary_)
displayStats()

Top 10 words for predicted phishing emails (considering Laplace smoothing): ['release', 'react', 'secret', 'countryside', 'returned', 'possibly', 'asset', 'attempting', 'scheme', 'abu']
Accuracy: 0.9554573274396464
Precision: 0.9812138728323699
Recall: 0.9282296650717703
F1-measure: 0.9539866526167895
ROC AUC score: 0.9553191627117985
Confusion Matrix:
 [[1452   26]
 [ 105 1358]]


In [17]:
ngrams = (1, 2)

vectorizer = CountVectorizer(min_df=2, lowercase=True, ngram_range=ngrams, stop_words='english', max_features=100000) ####
X_data_counts = vectorizer.fit_transform(balanced_X)

tf_transformer = TfidfTransformer(use_idf=True).fit(X_data_counts)
X_data_tf = tf_transformer.transform(X_data_counts)

X_train, X_test, y_train, y_test = train_test_split(X_data_tf, balanced_y, test_size=0.2, random_state=42)

classifier = MultinomialNB()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

In [18]:
topTenWords(vectorizer.vocabulary_)
displayStats()

Top 10 words for predicted phishing emails (considering Laplace smoothing): ['potential', 'bath', 'occur micro', 'indicating certain', 'paid', 'rx', 'imagination', 'applicant', 'operating', 'half year']
Accuracy: 0.9571574294457668
Precision: 0.9897435897435898
Recall: 0.9234449760765551
F1-measure: 0.9554455445544554
ROC AUC score: 0.9569863581330003
Confusion Matrix:
 [[1464   14]
 [ 112 1351]]


In [19]:
from sklearn.svm import SVC    ####

ngrams = (1, 1)

vectorizer = CountVectorizer(min_df=2, lowercase=True, ngram_range=ngrams, stop_words='english', max_features=500) ####
X_data_counts = vectorizer.fit_transform(balanced_X)

tf_transformer = TfidfTransformer(use_idf=True).fit(X_data_counts)
X_data_tf = tf_transformer.transform(X_data_counts)

X_train, X_test, y_train, y_test = train_test_split(X_data_tf, balanced_y, test_size=0.2, random_state=42)

classifier = SVC()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

displayStats()

Accuracy: 0.9585175110506631
Precision: 0.9472981987991995
Recall: 0.9706083390293917
F1-measure: 0.9588116137744767
ROC AUC score: 0.9585788650492019
Confusion Matrix:
 [[1399   79]
 [  43 1420]]


In [21]:
ngrams = (1, 5)

vectorizer = CountVectorizer(min_df=2, lowercase=True, ngram_range=ngrams, stop_words='english', max_features=100000) ####
X_data_counts = vectorizer.fit_transform(balanced_X)

tf_transformer = TfidfTransformer(use_idf=True).fit(X_data_counts)
X_data_tf = tf_transformer.transform(X_data_counts)

X_train, X_test, y_train, y_test = train_test_split(X_data_tf, balanced_y, test_size=0.2, random_state=42)

classifier = SVC()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

displayStats()

Accuracy: 0.976878612716763
Precision: 0.9684351914036265
Recall: 0.9856459330143541
F1-measure: 0.9769647696476964
ROC AUC score: 0.9769231018251743
Confusion Matrix:
 [[1431   47]
 [  21 1442]]


In [21]:
# pip install tensorflow

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv("Phishing_Email.csv", encoding = "latin-1")

X_data = data['Email Text']
y = data['Email Type']

import nltk

import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import re

def clean_text(text):    
  if not isinstance(text, str):
    text = str(text)  # to string
    
  number_pattern = r"\d+"
  text = re.sub(number_pattern, "", text)  # remove numbers
    
  url_pattern = r"http[s]?://\S+"  # remove URLs
  text = re.sub(url_pattern, "", text)
    
  text = text.lower()

  text = text.translate(str.maketrans('', '', string.punctuation)) #removes punctuation

  stop_words = stopwords.words('english')
  text = ' '.join([word for word in text.split() if word not in stop_words]) #removes stopwords

  lemmatizer = WordNetLemmatizer()
  text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

  return text

clean_X_data = []
for text in X_data:
  clean_X_data.append(clean_text(text))

from random import random
from collections import Counter

def balance_data(X_data, y, target_label="Safe Email", keep_proportion=0.64):
  class_counts = Counter(y)
  safe_count = class_counts[target_label]

  # keep 64% of the Safe Emails
  undersampled_data = []
  undersampled_labels = []
  for i, (data_point, label) in enumerate(zip(X_data, y)):
    if label == target_label:
      if random() < keep_proportion:
        undersampled_data.append(data_point)
        undersampled_labels.append(0)
    else:
      # keep all Phishing Emails
      undersampled_data.append(data_point)
      undersampled_labels.append(1)

  return undersampled_data, undersampled_labels

balanced_X, balanced_y = balance_data(clean_X_data, y)

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

ngrams = (1, 1)

vectorizer = CountVectorizer(min_df=2, lowercase=True, ngram_range=ngrams, stop_words='english', max_features=500) ####
X_data_counts = vectorizer.fit_transform(balanced_X)

tf_transformer = TfidfTransformer(use_idf=True).fit(X_data_counts)
X_data_tf = tf_transformer.transform(X_data_counts)

X_train, X_test, y_train, y_test = train_test_split(X_data_tf, balanced_y, test_size=0.2, random_state=42)

# Define the model architecture
model = keras.Sequential([
#  layers.Dense(8, activation="relu", input_shape=(X_train.shape[1],)),  # First hidden layer with 128 neurons and ReLU activation
#  layers.Dense(4, activation="relu"),  # Second hidden layer with 64 neurons and ReLU activation
#  layers.Dense(2, activation="sigmoid")  # Output layer with softmax activation (adapt the number of neurons to your number of classes)
])

# Compile the model
#model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
#model_history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on test data (optional)
#test_loss, test_acc = model.evaluate(X_test, y_test)
#print("Test accuracy:", test_acc)